In [ ]:
import os
import torch as tr
import numpy as np
from matplotlib import pyplot as plt
import itertools

from sklearn.metrics.pairwise import cosine_similarity
import time

from utilsEM import *

%load_ext autoreload
%autoreload 2

In [ ]:
tstamp = time.perf_counter_ns()

In [ ]:
neps = 10 # 10 
tsteps = 45 # 45

argsD = {
  'smtemp':8,
  'stim_weight':0.05,
  'hrate':0.04
}
mtag = "-".join(["%s_%s"%(i,j) for i,j in argsD.items()])

acc,score = run_EMexp(neps,tsteps,argsD)

### performance metrics 

In [ ]:
hits_stderr = np.concatenate((score.mean(2).std(-1)/np.sqrt(neps))[:,(0,1)])
correj_stderr = np.concatenate((score.mean(2).std(-1)/np.sqrt(neps))[:,(2,3)])
d,s = paper_dprime(
  np.concatenate(score.mean(2)[:,(0,1)]),
  np.concatenate(score.mean(2)[:,(2,3)])
)
print(d.shape,s.shape)
dprime_stderr = d.std(-1)/np.sqrt(neps)
bias_stderr = s.std(-1)/np.sqrt(neps)

In [ ]:
# 2back-target, 2back-lure, 3back-target, 3back-lure
hits = np.concatenate(acc[:,(0,1)]) 
correj = np.concatenate(acc[:,(2,3)])
dprime = np.zeros(4)
bias = np.zeros(4)
for i in range(4):
  d,s = paper_dprime(hits[i], 1-correj[i])
  dprime[i]=d
  bias[i]=s


In [ ]:
f,axar = plt.subplots(2,2,figsize=(15,8));axar=axar.reshape(-1)
cL = ['blue','darkblue','lightgreen','forestgreen']
labL = ['2b,ctrl','2b,lure','3b,ctrl','3b,lure']

# correct reject
ax = axar[0]
ax.set_title('correct rejection')
ax.bar(range(4),correj,color=cL,yerr=correj_stderr)

# hits
ax = axar[1]
ax.set_title('hits')
ax.bar(range(4),hits,color=cL,yerr=hits_stderr)

# 
ax = axar[2]
ax.set_title('dprime')
ax.bar(range(4),dprime,color=cL,yerr=dprime_stderr)

#
ax = axar[3]
ax.set_title('bias')
ax.bar(range(4),bias,color=cL,yerr=bias_stderr)

## 
for ax in axar[:2]:
  ax.set_xticks(np.arange(4))
  ax.set_xticklabels(labL)
  ax.set_ylim(0,1)

plt.savefig('figures/EMmetrics-%s-t%i.jpg'%(mtag,tstamp))
plt.savefig('figures/EMmetrics_yerr-%s-t%i.svg'%(mtag,tstamp))